In [ ]:
#loading dataframe
import pandas as pd
import numpy as np
df=pd.read_excel('Energy Indicators.xls', skiprows=17, skipfooter=38,usecols=range(2,6))
df.columns=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
df

In [ ]:
#Question 1 
#Remove numbers or parenthesis
df['Country']=df['Country'].str.replace('\d+| \(.*\)','')
# df.iloc[99]
#df.iloc[104]

#rename countries
countries_to_rename={'Republic of Korea':'South Korea','United States of America':'United States','United Kingdom of Great Britain and Northern Ireland':'United Kingdom','China, Hong Kong Special Administrative Region':'Hong Kong'}

for old_name, new_name in countries_to_rename.items():
    df['Country'].loc[df['Country'] == old_name] = new_name

df

In [ ]:
GDP=pd.read_csv('world_bank.csv', skiprows=3)

countries_to_rename={'Korea, Rep.':'South Korea','Iran, Islamic Rep.':'Iran','Hong Kong SAR, China':'Hong Kong'}

for old_name, new_name in countries_to_rename.items():
    GDP['Country Name'].loc[GDP['Country Name'] == old_name] = new_name

GDP

In [ ]:
ScimEn=pd.read_excel('scimagojr.xlsx')
ScimEn

In [ ]:
#this method allow to select multiple columns from GDP; In fact i dont need the first columns so i can do something a lot simpler. See next code block
(ScimEn.head(15)
        .set_index('Country')
        .merge(df.set_index('Country'), how='left', left_index=True, right_index=True)
        .merge(
                GDP.iloc[:, [0,1,2,3] + list(range(GDP.columns.get_loc('2006'),GDP.columns.get_loc('2015')+1))]
                    .set_index('Country Name'),
                how='left', left_index=True, right_index=True)
)



In [ ]:
def answer_one():
    return (ScimEn.head(15)
                  .set_index('Country')
                  .merge(df.set_index('Country'), how='left', left_index=True, right_index=True)
                  .merge(GDP.set_index('Country Name').loc[:,'2006':'2015'],how='left', left_index=True,                     right_index=True)
            )
answer_one()


In [ ]:
#question 3 : What is the average GDP over the last 10 years for each country? 
dfm= answer_one()
def answer_three():
 return dfm.loc[:,'2006':'2015'].mean(axis=1).sort_values(ascending=False)
avgGDP=answer_three()
avgGDP

In [ ]:
#By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?
def answer_four():
    # return (dfm[dfm.index == avgGDP.index[5]].loc[:,'2015']-dfm[dfm.index == avgGDP.index[5]].loc[:,'2006']).iloc[0]
    return (dfm.loc[:,'2015']-dfm.loc[:,'2006']).loc[avgGDP.index[5]]
answer_four()

In [ ]:
# What is the mean `Energy Supply per Capita`?
def answer_five():
    return dfm['Energy Supply per Capita'].mean()
answer_five()

In [ ]:
# What country has the maximum % Renewable and what is the percentage?
def answer_six():
    return (dfm['% Renewable'].idxmax(),dfm['% Renewable'].max())
answer_six()

In [ ]:
# Create a new column that is the ratio of Self-Citations to Total Citations. What is the maximum value for this new column, and what country has the highest ratio?
def answer_seven():
    dfm['self_to_total']=dfm['Self-citations']/dfm['Citations']
    return (dfm['self_to_total'].idxmax(),dfm['self_to_total'].max())
answer_seven()


In [ ]:
# Create a column that estimates the population using Energy Supply and Energy Supply per capita. What is the third most populous country according to this estimate?
def answer_eight():
    dfm['pop']=(dfm['Energy Supply']/dfm['Energy Supply per Capita'])*(10**6)
    return dfm['pop'].sort_values(ascending=False).index[2]
answer_eight()

In [ ]:
# Create a column that estimates the number of citable documents per person. What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the .corr() method, (Pearson's correlation).

def answer_nine():
    dfm['cit_by_pers']=dfm['Citable documents']/dfm['pop']
    return dfm.loc[:,['cit_by_pers','Energy Supply per Capita']].astype('float').corr(method='pearson').iloc[0,1]

answer_nine()


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

dfm.loc[:,['cit_by_pers','Energy Supply per Capita']].astype('float').plot(x='cit_by_pers',y='Energy Supply per Capita',kind='scatter',xlim=[0, 0.0006])



In [ ]:
# Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.
def answer_ten():
    dfm['High Renew'] = (dfm['% Renewable']> dfm['% Renewable'].median()).astype('int')
    return dfm['High Renew']

answer_ten()

In [ ]:
# Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
def answer_eleven():
    return (dfm[['pop']]
        .astype('float')
        .merge(pd.DataFrame.from_dict(ContinentDict, orient='index', columns=['Continent']),how='left',                left_index=True, right_index=True)
        .pivot_table(values='pop',index='Continent', aggfunc={'pop':[np.count_nonzero,np.sum, np.mean,np.std]})
        .set_axis(['size', 'sum', 'mean', 'std'], axis=1, inplace=False)
)
answer_eleven()

In [ ]:
# same with groupby
(dfm[['pop']]
    .astype('float')
    .merge(pd.DataFrame.from_dict(ContinentDict, orient='index', columns=['Continent']),how='left',                left_index=True, right_index=True)
    .set_index('Continent')
    .sort_index()
    .groupby(level=0)
    .agg(size=('pop', 'count'), sum=('pop', 'sum'),mean=('pop', 'mean'), std=('pop', 'std'))
)


In [ ]:
# Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

dfm2 = (dfm
    .merge(pd.DataFrame.from_dict(ContinentDict, orient='index', columns=['Continent']),how='left',                left_index=True, right_index=True)
    .reset_index()
    .set_index(['Continent', 'Country'])
    .sort_index()
)
pd.cut(dfm2['% Renewable'],5) 

In [ ]:
# Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

dfm2.style.format({'pop':'{:,}'})

In [ ]:
%matplotlib inline

Top15 = answer_one()
ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")